# Assessment 2: Machine Learning and Optimisation

## Part 1 – OPTIMISATION

### Task 2.1 – Generation of random solutions

#### Importing libraries

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [11]:
df = pd.read_table("Modules.txt")
df.head()
df.tail()

df.shape
df.columns

Index(['MOD001|Zacharias Karstensen|2|MOD002,MOD003,MOD004,MOD005,MOD006,MOD007,MOD008,MOD009,MOD010,MOD013'], dtype='object')

In [7]:
def onemax(x):
  return x.sum()

In [8]:
def generate_binary(D):
  ## Generate a array of D bits
  return np.random.randint(0, 11, D)

for i in range(5):
  x = generate_binary(10)
  y = onemax(x)

  print(f"{x} + {y}")

[ 7  5 10  8  3  3  4 10  7 10] + 67
[ 8  0  7  6  2  4  5  5 10  7] + 54
[9 8 9 2 9 6 5 5 1 7] + 61
[10  8 10  7  4  0  7  6 10  8] + 70
[2 7 3 2 5 9 3 9 5 0] + 45


### Task 2.2 – Algorithm implementation

### Task 2.3 – Visualisation of results